# Advanced Python
## `pandas` practice with Olist

In this notebook, you can see the solution of various non-trivial tasks with the use of module `pandas`.

In [1]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Data

In this assignment, you will have to step into the shoes of an analyst in a Brazilian marketplace [Olist](https://olist.com/pt-br/). You need to examine the data and from it draw conclusions that will help the business flourish!

The data will be downloaded in `archive.zip` after the next cell is executed.

In [2]:
# !pip install wget

In [2]:
# This code that will be in each test, do not change names of variables
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Van is given 9 datasets that contain all the data for 100,000 orders from all over Brazil. To make your life easier, here are the links to these datasets (the file `product_category_name_translation` is a translation of the category names from Portuguese to English)

![](https://i.imgur.com/HRhd2Y0.png)

All right, let's get started!

### Task №1

Determine:

* Number of items

* Average item price (item price = average of prices in items dataset)

by category (all categories must be in English)

The table you should have:

```
category | products | price

value    | value    | value
```

**Note:**

For the category `portateis_cozinha_e_preparadores_de_alimentos` translation is `portable kitchen and food preparers`

For the category `pc_gamer`, translation `PC Gamer`.

You need to add a translation for them separately.

In [3]:
%%write_and_run task_1.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_1(translation, items, products):
    # creating a dataframe with additional translations to add them
    translation_to_add = pd.DataFrame(
            [['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'], 
             ['pc_gamer', 'PC Gamer']],
        columns=translation.columns)
    # supplementing the dataframe with translations, so that I can join everything I need at once
    translation_full = pd.concat([translation, translation_to_add], ignore_index=True)
    # checking that there are 2 more categories in the translations
    assert translation_full.shape[0] == translation.shape[0] + 2

    # counting the average price of products
    items_price = items.groupby('product_id').agg({'price': 'mean'}).reset_index()
    # join aggregate info on products and aitems
    products_and_items = pd.merge(products, items_price, how='inner', on='product_id')
    # add translation. LEFT JOIN, because the product_category_name field is NULL
    products_items_and_translation = pd.merge(products_and_items, translation_full, how='left', on='product_category_name')
    # final aggregations
    res = products_items_and_translation.groupby('product_category_name_english').agg({'product_id': 'nunique', 'price': 'mean'}).reset_index()
    # rename as required
    res = res.rename({'product_category_name_english': 'category',
                      'product_id': 'products'}, axis=1)
    
    return res

In [4]:
task_1(translation, items, products)

,category,products,price
0,PC Gamer,3,195.996667
1,agro_industry_and_commerce,74,438.692969
2,air_conditioning,124,205.868773
3,art,55,195.019858
4,arts_and_craftmanship,19,65.839474
...,...,...,...
68,stationery,849,103.571095
69,tablets_printing_image,9,163.684210
70,telephony,1134,83.826234
71,toys,1411,133.278735


In [5]:
# Checks

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

### Task №2

Identify for each salesperson the main category of their sales (categories should be in English)

The table you should have:

```
seller_id | category

value     | value
```

In [6]:
%%write_and_run task_2.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    # creating a dataframe with additional translations to add them
    translation_to_add = pd.DataFrame(
            [['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'], 
                ['pc_gamer', 'PC Gamer']],
        columns=translation.columns)
    # supplementing the dataframe with translations, so that I can join everything I need at once
    translation_full = pd.concat([translation, translation_to_add], ignore_index=True)
    # combine products and category translations
    translation_full_products = pd.merge(products, translation_full, how='left', on='product_category_name')
    # merging the last dataset and the aitems
    items_translation_full_products = pd.merge(items, translation_full_products, how='inner', on='product_id')
    # group by seller_id and product_category_name_english, count the number of orders, 
    # sort by order count within a category
    grouped = items_translation_full_products.groupby(['seller_id', 
                                                       'product_category_name_english']
                                                       ).agg({"order_item_id": "count"}).reset_index().sort_values('order_item_id', ascending=False)
    # determine the most popular category for each seller_id
    res = grouped.groupby('seller_id').agg({'product_category_name_english': 'first'}).reset_index()
    # renaming it according to the assignment
    res = res.rename({'product_category_name_english': 'category'}, axis=1)

    return res

In [7]:
task_2(translation, products, items)

,seller_id,category
0,0015a82c2db000af6aaaf3ae2ecb0532,small_appliances
1,001cca7ae9ae17fb1caed9dfb1094831,garden_tools
2,001e6ad469a905060d959994f1b41e4f,sports_leisure
3,002100f778ceb8431b7a1020ff7ab48f,furniture_decor
4,004c9cd9d87a3c30c522c48c4fc07416,bed_bath_table
...,...,...
3030,ffcfefa19b08742c5d315f2791395ee5,books_general_interest
3031,ffdd9f82b9a447f6f8d4b91554cc7dd3,housewares
3032,ffeee66ac5d5a62fe688b9d26f83f534,home_appliances
3033,fffd5413c0700ac820c7069d66d98c89,housewares


In [8]:
# Checks

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

### Task №1

Print the percentage of money spent by each state (money spent - sum of money by orders delivered, sum by price and freight_value)

*Note:* State breakdown is by buyer's state, percentage is a number between 0 and 1


The table you should have:

```
state | perc

value | value
```

In [9]:
%%write_and_run task_3.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    # take only orders that have been delivered
    delivered_orders = orders[orders['order_status'] == 'delivered']
    # merge delivered orders with aitems
    delivered_orders_items = pd.merge(delivered_orders, items, 'inner', 'order_id')
    # merge past dataset with clients
    delivered_orders_items_customers = pd.merge(delivered_orders_items, customers)
    # group info by state 
    grouped_data = delivered_orders_items_customers.groupby('customer_state').agg({"price": "sum", "freight_value": "sum"}).reset_index()
    # get the total amount of orders: item price + shipping
    grouped_data['total_amt'] = grouped_data['price'] + grouped_data['freight_value']
    # get a share of every state
    grouped_data['perc'] =  grouped_data['total_amt'] / grouped_data['total_amt'].sum()
    # rename it and take only the columns I need
    res = grouped_data.rename({'customer_state': 'state'}, axis=1)
    res = res[['state', 'perc']]

    return res

In [10]:
task_3(orders, customers, items)

,state,perc
0,AC,0.001269
1,AL,0.006107
2,AM,0.001789
3,AP,0.001047
4,BA,0.038336
5,CE,0.017279
6,DF,0.022447
7,ES,0.020601
8,GO,0.021674
9,MA,0.009585


In [11]:
# Checks

res = task_3(orders, customers, items)
assert np.allclose(res.perc.sum(), 1)  # in case of numerical problems
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

### Task №4

Determine the average purchase receipt (add a breakdown for the cost of the order itself and the cost of shipping) and the average number of items in the order.

Also determine the average number of purchases per user (note the identifiers).

In [12]:
%%write_and_run task_4.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    # merge orders and items
    orders_items = pd.merge(orders, items, 'inner', 'order_id')
    # aggregate info, get sum of price, shipping cost, number of items in the order
    grouped_ord = orders_items.groupby('order_id').agg({'price': 'sum', 'freight_value': 'sum', 'order_item_id': 'count'}).reset_index()
    # counting the average price
    price_mean = grouped_ord['price'].mean()
    # calculating the average shipping cost
    freight_value_mean = grouped_ord['freight_value'].mean()
    # calculate the average number of items in an order
    order_item_id_mean = grouped_ord['order_item_id'].mean()
    
    # combining orders and customers
    orders_customers = pd.merge(orders, customers, 'inner', 'customer_id')
    # group by customer_unique_id, count the number of unique customer_id
    customers_data = orders_customers.groupby('customer_unique_id').agg({'customer_id': 'nunique'}).reset_index()
    # calculate the average number of purchases per user
    customer_id_mean = customers_data['customer_id'].mean()

    return price_mean, freight_value_mean, order_item_id_mean, customer_id_mean

In [13]:
task_4(items, orders, customers)

(137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

In [14]:
# Checking

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

### Task №5

Calculate CSAT (customer satisfaction - average review score) and display the average CSAT by day from April 2017 to April 2018.

All date manipulation should be done using `datetime` and `dateutil`.

The table you should get:

```
     date    | csat

"YYYY-MM-DD" | value
```

In [15]:
%%write_and_run task_5.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime as dt

def task_5(reviews):
    # make a copy of the dataset so I don't have to change the original dataset
    reviews_copy = reviews.copy()
    # set the date format YYYYY-mm-dd
    format = '%Y-%m-%d'
    # setting the right boundaries on the dates
    date_start = parse('2017-04-01', dayfirst=False)
    date_finish = parse('2018-04-30', dayfirst=False)
    # date parsing: add a column with date in datetime format
    reviews_copy['date_datetime'] = reviews_copy['review_creation_date'].apply(lambda date: parse(date, dayfirst=False))
    # filter only the period we need from date_start to date_finish
    pre_need_reviews = reviews_copy[date_start <= reviews_copy['date_datetime']]
    need_reviews = pre_need_reviews[pre_need_reviews['date_datetime'] <= date_finish]
    # translate datetime into a string in the required format 
    need_reviews['date'] = need_reviews['date_datetime'].apply(lambda date: dt.datetime.strftime(date, format))
    # aggregate by day, average CSAT, rename the column
    res = need_reviews.groupby('date').agg({'review_score': 'mean'}).reset_index()
    res = res.rename({'review_score': 'csat'}, axis=1)

    return res

In [18]:
task_5(reviews)

,date,csat
0,2017-04-01,4.200000
1,2017-04-02,2.857143
2,2017-04-03,5.000000
3,2017-04-04,4.247934
4,2017-04-05,4.035088
...,...,...
382,2018-04-26,4.188272
383,2018-04-27,4.016077
384,2018-04-28,4.043887
385,2018-04-29,3.688172


In [19]:
# Checks
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

### Task №6

Look at how fast users respond (do aggregation by the number of days of response) and what the average score is.

All time manipulation should be done via `datetime` and `dateutil`.

The table you should have:

```
days  | csat  | orders

value | value | value
```

The results should be sorted by day in ascending order.

In [20]:
%%write_and_run task_6.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime as dt

def task_6(reviews):
    # make a copy of the dataset so I don't have to change the original dataset
    reviews_copy = reviews.copy()

    # parsing of review and response publication dates
    reviews_copy['review_creation_date_datetime'] = reviews_copy['review_creation_date'].apply(lambda date: parse(date, dayfirst=False))
    reviews_copy['review_answer_timestamp_datetime'] = reviews_copy['review_answer_timestamp'].apply(lambda date: parse(date, dayfirst=False))
    # calculate the delta between publication and response
    reviews_copy['delta'] = reviews_copy['review_answer_timestamp_datetime'] - reviews_copy['review_creation_date_datetime']
    # convert delta to number of days
    reviews_copy['delta_days'] = reviews_copy['delta'].apply(lambda date: date.days)
    # aggregate by response day: average CSAT and number of responses
    res = reviews_copy.groupby('delta_days').agg({'review_score': 'mean', 'delta': 'count'}).reset_index()
    # rename the columns
    res = res.rename({'delta_days': 'days', 'review_score': 'csat', 'delta': 'orders'}, axis=1)

    return res

In [21]:
task_6(reviews)

,days,csat,orders
0,0,3.886663,24361
1,1,4.235586,30995
2,2,4.048685,15898
3,3,4.194567,14062
4,4,4.042382,4601
...,...,...,...
209,446,5.000000,1
210,471,4.000000,1
211,508,5.000000,1
212,512,5.000000,1


In [22]:
# Checks

res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

### Task №7

Highlight all orders where the order_delivered_customer_date field is not set. Replace it with the date '2999-12-31'

In [23]:
%%write_and_run task_7.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_7(orders):
    # I make a copy of the dataset so I don't have to change the original dataset
    orders_copy = orders.copy()
    # fill NaN 2999-12-31
    orders_copy['order_delivered_customer_date'] = orders_copy['order_delivered_customer_date'].fillna('2999-12-31')

    return orders_copy

In [24]:
task_7(orders)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [25]:
# Checks

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

### Task №8

Identify the top 10 sellers with more than 100 orders who most frequently ship their package to other regions (only shipped orders count)

Most frequently shipped = highest percentage of orders shipped to another state

The table you should have:

```
seller_id | share

  value   | value
```

In [35]:
%%write_and_run task_8.py

# important! all dependencies that you use (if you add new ones) in this class must be explicitly duplicated in this cell
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    # merge sellers, items, orders and customers
    sellers_items = pd.merge(sellers, items, 'inner', 'seller_id')
    orders_sellers_items = pd.merge(orders, sellers_items, 'inner', 'order_id')
    data = pd.merge(customers, orders_sellers_items, 'inner', on='customer_id')
    # count the number of unique orders from the sellers
    sellers_data = data.groupby('seller_id').agg({"order_id": "nunique"}).reset_index()
    # take sellers with more than 100 orders
    seller_id_needed = sellers_data[sellers_data['order_id'] > 100]
    # taking info on the right sellers
    sellers_needed = sellers[sellers['seller_id'].isin(seller_id_needed['seller_id'].values)]
    # merge the right sellers and delivered orders
    sellers_needed_items = pd.merge(sellers_needed, items, 'inner', 'seller_id')
    orders_delivered = orders[orders['order_status'] == 'delivered']
    orders_sellers_needed_items = pd.merge(orders_delivered, sellers_needed_items, 'inner', 'order_id')
    # add customer info
    data_needed = pd.merge(customers, orders_sellers_needed_items, 'inner', on='customer_id')
    # make a flag: the sender and the customer are from different states
    data_needed['is_other_state'] = (data_needed['seller_state'] != data_needed['customer_state']).astype(int)

    # count the proportion of orders with different states of shipper and customer
    res = data_needed.drop_duplicates('order_id').groupby('seller_id').agg({"is_other_state": "mean"}).reset_index()
    res = res.rename({'is_other_state': 'share'}, axis=1).sort_values('share', ascending=False)
    res = res.iloc[:10].reset_index(drop=True)

    return res

In [36]:
task_8(orders, items, sellers, customers)

,seller_id,share
0,001cca7ae9ae17fb1caed9dfb1094831,0.974359
1,44073f8b7e41514de3b7815dd0237f4f,0.971831
2,06a2c3af7b3aee5d69171b0e14f0ee87,0.964010
3,a3a38f4affed601eb87a97788c949667,0.956000
4,de722cd6dad950a92b7d4f82673f8833,0.946429
5,1b4c3a6f53068f0b6944d2d005c9fc89,0.945946
6,aaed1309374718fdd995ee4c58c9dfcd,0.942308
7,1c68394e931a64f90ea236c5ea590300,0.942029
8,c33847515fa6305ce6feb1e818569f13,0.938596
9,7178f9f4dd81dcef02f62acdf8151e01,0.935644


In [37]:
# Checks

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.517151493824779